In [12]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.optim as optim
import torchmetrics
import torch.nn.functional as F
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import time
import torchvision
from torchvision import transforms
from torchvision import datasets
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer
from tqdm.notebook import tqdm
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available:
    device_name = torch.cuda.get_device_name(0)
    print(device_name)
print(f"Using device: {device}")

NVIDIA GeForce RTX 4050 Laptop GPU
Using device: cuda:0


In [13]:
# Load training data
with open('/home/csalitre/school/ecgr-5106/intro-to-deeplearning/Datasets/sample_data.txt', 'r') as file:
    text = file.read()
    print(text)

“Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting the next character in a sequence of text based on the characters that precede it. This task is essential for various applications, including text auto-completion, spell checking, and even in the development of sophisticated AI models capable of generating human-like text.

At its core, next character prediction relies on statistical models or deep learning algorithms to analyze a given sequence of text and predict which character is most likely to follow. These predictions are based on patterns and relationships learned from large datasets of text during the training phase of the model.

One of the most popular approaches to next character prediction involves the use of Recurrent Neural Networks (RNNs), and more specifically, a variant called Long Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for sequential data like text, as they can mainta

In [14]:
# PREPROCESSING

# remove duplicate characters and sorts the list
chars = sorted(list(set(text)))

# create a dictionary that maps each character to unique index(integer)
ix_to_char = {i: ch for i, ch in enumerate(chars)}
# Similar to previous line, but in reverse
char_to_ix = {ch: 1 for i, ch in enumerate(chars)}
chars = sorted(list(set(text)))

# preparing dataset
max_length = 30     # maximum length of input sequences
x = []
y = []

for i in range(len(text) - max_length):
    sequence = text[i:i + max_length]
    label = text[i + max_length]
    x.append([char_to_ix[char] for char in sequence])
    y.append(char_to_ix[label])

x = np.array(x)
y = np.array(y)

# Splitting the dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# Converting data to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)
x_val = torch.tensor(x_val, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)

# Defining the RNN model
class CharLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharLSTM, self).__init__()
        self.hidden_size = hidden_size
        #This line takes the input tensor x, which contains indices of characters, and passes it through an embedding layer (self.embedding). 
        #The embedding layer converts these indices into dense vectors of fixed size. 
        #These vectors are learned during training and can capture semantic similarities between characters. 
        #The result is a higher-dimensional representation of the input sequence, where each character index is replaced by its corresponding embedding vector. 
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        #The RNN layer returns two outputs: 
        #1- the output tensor containing the output of the RNN at each time step for each sequence in the batch, 
        #2-the hidden state (_) of the last time step (which is not used in this line, hence the underscore).
        output, _ = self.rnn(embedded)
        #The RNN's output contains the outputs for every time step, 
        #but for this task, we're only interested in the output of the last time step because we're predicting the next character after the sequence. 
        #output[:, -1, :] selects the last time step's output for every sequence in the batch (-1 indexes the last item in Python).
        output = self.fc(output[:, -1, :])  # Get the output of the last RNN cell
        return output

# Hyperparameters
hidden_size = 128
learning_rate = 0.005
epochs = 100

# Model, loss, and optimizer
model = CharLSTM(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(x_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(x_val)
        val_loss = criterion(val_output, y_val)
        #The use of the underscore _ is a common Python convention to indicate that the actual maximum values returned by torch.max are not needed and can be disregarded. 
        #What we are interested in is the indices of these maximum values, which are captured by the variable predicted. These indices represent the model's predictions for each example in the validation set.
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]



Epoch 10, Loss: 0.004570871125906706, Validation Loss: 0.0027736066840589046, Validation Accuracy: 1.0
Epoch 20, Loss: 0.00018785615975502878, Validation Loss: 0.00015770144818816334, Validation Accuracy: 1.0
Epoch 30, Loss: 5.7218825531890616e-05, Validation Loss: 5.352351945475675e-05, Validation Accuracy: 1.0
Epoch 40, Loss: 3.58813522325363e-05, Validation Loss: 3.480850500636734e-05, Validation Accuracy: 1.0
Epoch 50, Loss: 2.9682674721698277e-05, Validation Loss: 2.9325052310014144e-05, Validation Accuracy: 1.0
Epoch 60, Loss: 2.7417760065873154e-05, Validation Loss: 2.7417761884862557e-05, Validation Accuracy: 1.0
Epoch 70, Loss: 2.6821739083970897e-05, Validation Loss: 2.6702529794420116e-05, Validation Accuracy: 1.0
Epoch 80, Loss: 2.6225696274195798e-05, Validation Loss: 2.6225696274195798e-05, Validation Accuracy: 1.0
Epoch 90, Loss: 2.5868070224532858e-05, Validation Loss: 2.5868075681501068e-05, Validation Accuracy: 1.0
Epoch 100, Loss: 2.5748884581844322e-05, Validation L

In [15]:
# Predicting the next character
test_str = "This is a simple example to demonstrate"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Predicted next character: ' '
